# Landscape Singlet Lens

## Landscape rear configuration

In [ ]:
import torch
import torchlensmaker as tlm

import matplotlib.pyplot as plt


def plot_magnification(optics, sampling):
    """
    Compute and plot magnification data for the given optical system
    The system must compute object and image coordinates
    """

    # TODO add color_dim

    # Evaluate the optical stack
    output = optics(tlm.default_input(dim=2, dtype=torch.float64, sampling=sampling))

    # Extract object and image coordinate (called T and V)
    T = output.rays_object
    V = output.rays_image

    mag, residuals = tlm.linear_magnification(T, V)

    fig, ax = plt.subplots(figsize=(12, 8))
    ax.plot(T.detach().numpy(), V.detach().numpy(), linestyle="none", marker="+")

    X = torch.linspace(T.min().item(), T.max().item(), 50)
    ax.plot(
        X.detach().numpy(),
        (mag * X).detach().numpy(),
        color="lightgrey",
        label=f"mag = {mag:.2f}",
    )

    ax.set_xlabel("Object coordinates")
    ax.set_ylabel("Image coordinates")
    ax.legend()

    plt.show()


In [ ]:
import torch
import torch.nn as nn
import torchlensmaker as tlm
import torch.optim as optim

# Setup two spherical surfaces with initial radiuses
surface1 = tlm.Sphere(diameter=30, r=tlm.parameter(torch.tensor(-55.)))
surface2 = tlm.Sphere(diameter=30, r=tlm.parameter(torch.tensor(-35.)))

lens = tlm.Lens(surface1, surface2, (1.0, 1.5), outer_thickness=3)

# Build the optical sequence
optics = nn.Sequential(
    tlm.ObjectAtInfinity(beam_diameter=10, angular_size=20),
    tlm.Gap(15),
    lens,
    tlm.Gap(120),
    tlm.ImagePlane(diameter=60),
)

tlm.show(optics, dim=2, end=200, sampling={"base": 10, "object": 5, "sampler": "uniform"}, color_dim="object")

plot_magnification(optics, sampling={"base": 10, "object": 5, "sampler": "uniform"})


In [ ]:
# Find the best parameters for the shapes
tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-3),
    sampling = {"base": 10, "object": 5, "sampler": "uniform"},
    dim = 2,
    num_iter = 200,
).plot()

# TODO always zero out y axis of loss plot

In [ ]:
tlm.show(optics, dim=2, end=200, sampling={"base": 10, "object": 5, "sampler": "uniform"}, color_dim="object")
plot_magnification(optics, sampling={"base": 10, "object": 5, "sampler": "uniform"})

## Landscape front configuration

In [ ]:
import torch
import torch.nn as nn
import torchlensmaker as tlm
import torch.optim as optim

# Setup two spherical surfaces with initial radiuses
surface1 = tlm.Sphere(diameter=30, r=tlm.parameter(torch.tensor(35.)))
surface2 = tlm.Sphere(diameter=30, r=tlm.parameter(torch.tensor(55.)))

lens = tlm.Lens(surface1, surface2, (1.0, 1.5), outer_thickness=3)

# Build the optical sequence
optics = nn.Sequential(
    tlm.ObjectAtInfinity(beam_diameter=10, angular_size=20),
    tlm.Gap(15),
    lens,
    tlm.Gap(120),
    tlm.ImagePlane(diameter=120),
)

tlm.show(optics, dim=2, end=200, sampling={"base": 10, "object": 5, "sampler": "uniform"}, color_dim="object")

#plot_magnification(optics, sampling={"base": 10, "object": 5, "sampler": "uniform"})

In [ ]:
# Find the best parameters for the shapes
tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-3),
    sampling = {"base": 10, "object": 5, "sampler": "uniform"},
    dim = 2,
    num_iter = 200,
).plot()

In [ ]:

tlm.show(optics, dim=2, end=200, sampling={"base": 10, "object": 10, "sampler": "uniform"}, color_dim="object")

#plot_magnification(optics, sampling={"base": 10, "object": 5, "sampler": "uniform"})

In [ ]:
from IPython.display import display
import build123d as bd

part = tlm.lens_to_part(lens)
display(part)